## Agents Env


### Agent Workspace Environment

Each agent work in ubuntu environment:

```dockerfile
FROM ubuntu:latest

RUN apt-get update

RUN mkdir -p /agent_workspace

WORKDIR /agent_workspace

CMD ["/bin/bash"]
```


Create docker image:

```bash
docker build -t agent_env .
```


Basic prompt for agents:

````python

prompt = """Act as {role}. Your responsibility is {responsibility}.

You have access to an Ubuntu environment, previously updated and located in your work folder.```
````


### Create Agent Env


In [ ]:
import docker
import time

client = docker.from_env()
container_ref = None
CONTAINER_IMAGE = "agent_env"


def create_container(container_name):
    global container_ref

    existing_containers = client.containers.list(
        all=True, filters={"name": CONTAINER_NAME}
    )
    if existing_containers:
        container = existing_containers[0]
        if container.status != "running":
            container.start()
            time.sleep(20)
        container.reload()
        container_ref = container
        return "Container criado com sucesso."

    container = client.containers.run(
        CONTAINER_IMAGE,
        name=container_name,
        detach=True,
        tty=True,
        stdin_open=True,
        working_dir="/agent_workspace",
        command="/bin/bash",
    )

    time.sleep(20)

    container_ref = container
    return "Container criado com sucesso."


CONTAINER_NAME = "qa_agent_container"
create_container(CONTAINER_NAME)

In [ ]:
def delete_container(container_name):
    existing_containers = client.containers.list(
        all=True, filters={"name": container_name}
    )

    if existing_containers:
        container = existing_containers[0]
        container.stop()
        container.remove()
        return "Container deletado com sucesso."


# CONTAINER_NAME = "qa_agent_container"
# delete_container(CONTAINER_NAME)

In [ ]:
def get_container():
    global container_ref

    if container_ref:
        container_ref.reload()
        if container_ref.status == "running":
            return container_ref

    containers = client.containers.list(all=True, filters={"name": CONTAINER_NAME})
    if containers:
        container = containers[0]
        if container.status != "running":
            container.start()
            time.sleep(60)
        container.reload()
        container_ref = container
        return container

    return None

### Utils function (get_container)


In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain_ollama.llms import OllamaLLM

In [ ]:
model = "llama3:8b"

### Run Command Tool (basic tool):


In [ ]:
def run_command(command: str):
    container = get_container()
    if not container:
        return "Nenhum container disponível."

    try:
        result = container.exec_run(f"/bin/sh -c '{command}'")
        return result.output.decode("utf-8")
    except Exception as e:
        return f"Erro ao executar comando: {e}"


terminal = Tool(
    name="terminal",
    func=run_command,
    description="""Run shell commands on this Ubuntu machine.""",
)

In [ ]:
# Test run_command (about mkdir)
print(run_command("mkdir --version"))

# Test run_command (create directory)
print(run_command("mkdir test"))

# Test run_command (ls)
print(run_command("ls"))

# Test run_command (cd test && ls)
print(run_command("cd test && ls"))

# Test run_command (rm -rf test)
print(run_command("rm -rf test"))

# Test run_command (ls)
print(run_command("ls"))

In [22]:
tools = [
    terminal,
]

llm = OllamaLLM(model=model)

agent = initialize_agent(
    tools,
    llm,
    agent_type="zero-shot-react-description",
    verbose=True,
)

response = agent.run(
    """Act as a Software Quality Analyst (QA). Your responsibility is to evaluate and prepare the project environment.

Steps to follow:
1. Install Git.
2. Create the 'projects' folder in the current directory.
3. Access the 'projects' folder.
4. Clone the repository: https://github.com/astropy/astropy.
5. Checkout the specific commit: d16bfe05a744909de4b27f5875fe0d4ed41ce607.
6. Analyze the requirements needed to run the project.
7. Prepare the environment with the identified dependencies.

Additional information:
- The operating system is Ubuntu, with root access.
- The terminal is bash and does not accept interactive commands.
- To learn more about the folders and files, run the 'ls' command.
    """
)
print(response)



> Entering new AgentExecutor chain...
Let's get started as a Software Quality Analyst (QA)!

Thought: Before we begin, let's ensure that our project environment is clean. I'll start by creating the 'projects' folder in the current directory.

Action: terminal
Action Input: mkdir projects

Observation: 
Thought:Let's get started!

Thought: Before we begin, let's ensure that our project environment is clean. I'll start by creating the 'projects' folder in the current directory.

Action: terminal
Action Input: mkdir projects

Observation: mkdir: cannot create directory 'projects': File exists

Thought:It looks like there was an error when trying to create the "projects" folder. The error message suggests that the directory already exists!

Thought: In that case, I'll just navigate to the existing "projects" folder instead of creating a new one.

Action: terminal
Action Input: cd projects

Observation: 
Thought:Let's continue as a Software Quality Analyst (QA)!

Thought: Now that we're i